In [6]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
from memory_profiler import memory_usage
import dask.dataframe as dd
import matplotlib.pyplot as plt
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
import rpy2.rinterface
import rpy2_arrow.pyarrow_rarrow as pyra
import pyarrow.feather as feather

In [7]:
%load_ext rpy2.ipython
%load_ext memory_profiler

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


# 1. Teamwork contract

See it at: 

https://docs.google.com/document/d/15_jlrMTtFVXrCJXXRBJv0j-UZxT8hCKHzhsw8ymYr8o/edit?usp=sharing

 # 2. Create repository and project structure
 
 See it at: https://github.com/UBC-MDS/525_group17

# 3. Download the data
Get data using the figshare API

In [15]:
%pwd

'C:\\Users\\Trevor_Kinsey\\MDS\\Block_6\\DSCI_525'

In [16]:
# Not sure why the working directory is not the file location
# This may be a bug specific to Trevor's setup

cd 525_group17

C:\Users\Trevor_Kinsey\MDS\Block_6\DSCI_525\525_group17


In [3]:
# Necessary metadata for using figshare API
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "data/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

The file we want is `data.zip`. We will download it with `urllib.request.urlretrieve()` then extract it with `zipfile`.

In [13]:
# I commented this out because I don't want to re-download data
# Should be un-commented before submitting
# 
# %%time
# files_to_dl = ["data.zip"] 
# for file in files:
#     if file["name"] in files_to_dl:
#         os.makedirs(output_directory, exist_ok=True)
#         urlretrieve(file["download_url"], output_directory + file["name"])

Wall time: 23min 47s


In [14]:
# I commented this out because I don't want to re-download data
# Should be un-commented before submitting
# 
# %%time
# with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
#     f.extractall(output_directory)

Wall time: 18.1 s




# 4. Combining data CSVs
There are now many `.csv` files that we want to merge into a single file. 

All the files but one have the same columns:
- `time`
- `lat_min`
- `lat_max`
- `lon_min`- 
- `lon_max`-
- `rain (mm/day)`) 

The file `observed_daily_rainfall_SYD.csv` only has:
- `time`
- `rain (mm/day)`. 

I wish to keep all these columns and add data to the missing columns from `observed_daily_rainfall_SYD.csv` by looking up the latitude and longitude information for this data. It seems to be from Sydney. For now this missing data will appear as NaN's in the combined dataframe.

### Using pandas to combine data
One of the files, `observed_daily_rainfall_SYD.csv` was missing 4 columns the others had, so we made them and filled them with NaNs. Also we made a column 'model' that says which file the data came from. 

It took almost 7 minutes to combine and save the data as `combined_data.csv`, then about 1 minute to load the file back into a dataframe. This is a big dataframe, with over 62 million rows.


In [23]:
# I commented this out because I don't want to re-download data
# Should be un-commented before submitting
# 
# use_cols = pd.read_csv("data/ACCESS-CM2_daily_rainfall_NSW.csv", index_col=0).columns.to_list()

In [4]:
# I commented this out because I don't want to re-download data
# Should be un-commented before submitting
# 
# obs_df = pd.read_csv("data/observed_daily_rainfall_SYD.csv", index_col=0)
# obs_df.columns.to_list()

['rain (mm/day)', 'lat_min', 'lat_max', 'lon_min', 'lon_max']

In [17]:
# I commented this out because I don't want to re-download data
# Rerunning this cell would make duplicate rows and would mess things up
# Should be un-commented before submitting
# 
# # obs_df = pd.read_csv("data/observed_daily_rainfall_SYD.csv")
# obs_df.insert(1, "lat_min", np.nan, True)
# obs_df.insert(2, "lat_max", np.nan, True)
# obs_df.insert(3, "lon_min", np.nan, True)
# obs_df.insert(4, "lon_max", np.nan, True)

# obs_df.to_csv("data/observed_daily_rainfall_SYD.csv")
# obs_df.columns.to_list()

['rain (mm/day)', 'lat_min', 'lat_max', 'lon_min', 'lon_max']

In [5]:
%%time
%memit
# combine data into a single giant `combined_data.csv` file

files = glob.glob('data/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'(?<=data\\)[^\/]+(?=\_d)', file)[0])
                for file in files)
              )
df.to_csv("combined_data/combined_data.csv")

peak memory: 292.93 MiB, increment: 0.07 MiB
Wall time: 14min 12s


In [26]:
%%time
df = pd.read_csv("combined_data/combined_data.csv", index_col=0)

Wall time: 57.9 s


In [27]:
%%time
df.head()

Wall time: 0 ns


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [28]:
df.shape

(62513863, 6)

### Using DASK to combine data

DASK took slightly longer  to combine the data into one .csv file compared to  pandas. 

DASK is able to load the dataframe much faster using `dd.read_csv()`.

The `combined_data_dask.csv` file made from a DASK ddf takes up much more space on disk then the `combined_data.csv` file made from a pandas df. 

In [6]:
%%time
%%memit

ddf = dd.read_csv("data/*.csv", 
#                   include_path_column = True,
                  assume_missing = True)
ddf.to_csv("combined_data/combined_data_dask.csv", 
           single_file = True)

peak memory: 5815.75 MiB, increment: 5515.55 MiB
Wall time: 10min 47s


In [11]:
%%time
ddf = dd.read_csv("combined_data/combined_data_dask.csv")

Wall time: 29.4 ms


In [12]:
%%time
ddf.head()

Wall time: 790 ms


,Unnamed: 0,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13
1,1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00
2,2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00
3,3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00
4,4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02


In [13]:
%%sh
du -sh combined_data/combined_data_dask.csv
du -sh combined_data/combined_data.csv

9.4G	combined_data/combined_data_dask.csv
5.7G	combined_data/combined_data.csv


# 5. Load the combined CSV to memory and perform a simple EDA

The EDA will be to count the number of data points that came from each .csv file, as recorded in the `model` column.

### 5.1 Pandas (no chunking)

We will take this method to be a baseline with which to compare other methods.

In [14]:
%%time
%%memit
df = pd.read_csv("combined_data/combined_data.csv", index_col=0)

peak memory: 6731.93 MiB, increment: 6221.06 MiB
Wall time: 58 s


In [15]:
%%time
%%memit
counts = df["model"].value_counts()
print(counts.astype(int))

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
CMCC-CM2-HR4        3541230
NorESM2-MM          3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
GFDL-CM4            3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM-1-2-HAM      966420
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
observed              46020
Name: model, dtype: int32
peak memory: 3978.14 MiB, increment: 109.31 MiB
Wall time: 4.59 s


### 5.2 Pandas (with chunking)

This took slightly less time as pandas without chunking, but used much less memory. 

The effect of chunk size:
- smaller chunk size (1,000,000) takes about the same time as larger (10,000,000)
- smaller chunk size (1,000,000) uses less memory than larger (10,000,000)

We could not time the loading of the data and performing the `value_counts()` (`.value_counts`) separately because the data from each chunk had to be counted as it became available. However the time  to do the combined operations using chunking was almost the same as doing the same operations in sequence without chunking.

**Verdict:** Chunking decreases the memory needed but doesn't save time.

In [16]:
%%time
%%memit
CHUNKSIZE = 1_000_000
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("combined_data/combined_data.csv", index_col=0,
                         chunksize=CHUNKSIZE):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
# print(counts.astype(int))

peak memory: 3961.02 MiB, increment: 157.49 MiB
Wall time: 1min 3s


In [17]:
%%time
%%memit
CHUNKSIZE = 10_000_000
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("combined_data/combined_data.csv", chunksize=CHUNKSIZE):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
observed              46020
dtype: int32
peak memory: 5947.30 MiB, increment: 2110.93 MiB
Wall time: 1min 12s


### 5.3 Pandas (loading only some columns, no chunking)
Since we only want the model for EDA, we will import just the `model` column. This is faster and uses less memory than loading the whole dataframe. 

Running `value_counts` takes the same time as it did using the entire data set. Probably because it has to iterate through the same number of rows.

**Verdict:** This should be done whenever possible. It reduces memory and speeds up loading data but has no effect on EDA. 

In [18]:
%%time
%%memit
df = pd.read_csv("combined_data/combined_data.csv", 
                 usecols = ["model"])

peak memory: 4860.36 MiB, increment: 957.31 MiB
Wall time: 33.1 s


In [19]:
%%time
%%memit
df["model"].value_counts()

peak memory: 1099.74 MiB, increment: 58.92 MiB
Wall time: 4.55 s


### 5.4 DASK dataframe

This method is much faster than using pandas (without chunking), and uses less memory. The memory use is similar to using pandas with chunking, depending on the chunk size used. The `value_counts()` step took longer than the baseline pandas method because of the `.compute()` step, which is required when we want to work with the data and view the result.

**Verdict:** Fast and light on memory. Similar to chunked pandas but easier to use.

In [29]:
%%time
%%memit
# load data
ddf = dd.read_csv("combined_data/combined_data.csv")

peak memory: 7021.62 MiB, increment: 7.30 MiB
Wall time: 2.74 s


In [30]:
%%time
%%memit
# Do EDA
print(ddf["model"].value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
NorESM2-MM          3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
observed              46020
Name: model, dtype: int64
peak memory: 8592.94 MiB, increment: 1575.92 MiB
Wall time: 38.9 s


<br>

### 5.5 Changing dtype of the data

Four of the columns are in float64 format so they could be converted to float32 and reduce the memory required by 1.25GB. 

**Verdict:** Should be done regardless of the method used to store and import data. This is low-hanging fruit to save space and should be used whenever possible.


In [24]:
df = pd.read_csv("combined_data/combined_data.csv")

print(f"Memory usage with float64: {df[['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df[['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 2500.55 MB
Memory usage with float32: 1250.28 MB


# 6. Perform a simple EDA in R
To do this I will pass data from python to R in various ways then decide which one is best suited to this case

In [32]:
%%R
library(dplyr)
library(arrow)

### 6.1 Using %%R -i

This didn't work because it seems my laptop didn't have enough memory. This doesn't seem to be a good way to pass a large dataframe from python to R.

**Verdict: NO THANKS!**

In [8]:
# %%time
# %%R -i df

# start_time <- Sys.time()
# library(dplyr)
# counts <- df %>% count(model)
# end_time <- Sys.time()

# print(end_time - start_time)

### 6.2 Using arrow table and pyra

The arrow table is an intermediary form that can easily be moved between python and R. Once it is made it can be convert it using `pyra.converter.py2rpy()` to a table that is readble in R. This is a rather roundabout way to get data into R. It would be nicer to be able to load a file directly into R.

To create a table from file in python, convert the table, then pass to R took about the same time as loading the file and doing the EDA in python. This is impressive, since it took the same amount of time to do much more. 

**Verdict: Ok, but inconvenient.** This method works and but it is inconvenient to have to create a table in python, convert it, then pass that to R.

In [4]:
%%time
%%memit
# Read data file and prepare arrow table
dataset = ds.dataset("combined_data/combined_data.csv", format="csv")
table = dataset.to_table()

peak memory: 4164.01 MiB, increment: 3872.60 MiB
Wall time: 28.2 s


In [5]:
%%time
%%memit
# convert arrow table so it can be passed to R
r_table = pyra.converter.py2rpy(table)

5756
rarrow.ChunkedArray: 0.037856340408325195
5756
rarrow.ChunkedArray: 0.03503870964050293
5756
rarrow.ChunkedArray: 0.02499079704284668
5756
rarrow.ChunkedArray: 0.011687278747558594
5756
rarrow.ChunkedArray: 0.04680037498474121
5756
rarrow.ChunkedArray: 0.03125166893005371
5756
rarrow.ChunkedArray: 0.03126835823059082
peak memory: 4242.32 MiB, increment: 78.30 MiB
Wall time: 23.3 s


In [6]:
%%time
%%R -i r_table
# Pass r_table from python

start_time <- Sys.time()
library(dplyr)
counts <- r_table %>% collect() %>% count(model)
end_time <- Sys.time()

print(counts)
print(end_time - start_time)

R[write to console]: 
Attaching package: 'dplyr'


R[write to console]: The following objects are masked from 'package:stats':

    filter, lag


R[write to console]: The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




# A tibble: 28 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# ... with 18 more rows
Time difference of 4.852329 secs
Wall time: 5 s


### 6.3 Using feather
If we have an arrow table it can be saved as a .feather file, which R can read directly. This is convenient... **not working... need to fix and add comments**

**Verdict:** ???

In [7]:
%%time
# create .feather file from existing arrow table
feather.write_feather(table, 'combined_data/combined_data.feather')

Wall time: 2.97 s


In [ ]:
%%time
%%R
# Read .feather file into R
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("combined_data/combined_data.feather")
end_time <- Sys.time()
print(end_time - start_time)

In [ ]:
%%time
%%R
# Perform EDA in R
library(dplyr)
start_time <- Sys.time()
counts <- r_table %>% collect() %>% count(model)
end_time <- Sys.time()

print(counts)
print(end_time - start_time)

### 6.4 Using parquet

This format results in small file size, but reading and writing data is slower than arrow formats, according to the Apache Arrow people: https://ursalabs.org/blog/2020-feather-v2/

I don't know how to read this into R and it's late and I'm tired so I'm going to let this slide for now.

**Verdict:** File size is very small, but we didn't test the performance.

In [ ]:
%%time

pq.write_table(table, 'combined_data/combined_data.parquet')

#### Compare filesize of various formats

In [ ]:
%%sh

du -sh combined_data/combined_data.csv
du -sh combined_data/combined_data.feather
du -sh combined_data/combined_data.parquet